In [1]:
import pandas as pd
import numpy as np
import math
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import dateutil
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, mean_absolute_error
import plotly.plotly as py
import plotly.graph_objs as go
from scipy import stats

# Carga y limpieza de datos

In [2]:
df_start = pd.read_excel('../Data/Graficas DustTrack Honeywell 5sept comp.xlsx')
df_start.head()

,Valor DustTrack,Valor Honeywell,Hora,Unnamed: 3
0,26,10,07:06:26,NaN
1,26,10,07:06:27,NaN
2,26,9,07:06:28,NaN
3,25,9,07:06:29,NaN
4,22,9,07:06:30,NaN


In [3]:
# Eliminar nulos y headers repetidos

df_start = df_start[~df_start['Valor DustTrack'].isna()]
df_start = df_start[df_start.columns.values[:-1]]
sum(df_start['Valor DustTrack'] == 'Valor DustTrack')
df_start = df_start[np.logical_not(df_start['Valor DustTrack'] == 'Valor DustTrack')]

df_start.index = np.arange(len(df_start))

In [4]:
# Casting a variables numericas

df_start[['Valor DustTrack', 'Valor Honeywell']] = df_start[['Valor DustTrack', 'Valor Honeywell']].apply(pd.to_numeric)

In [5]:
def plot_time_series(df, variables, filename):    
    data = list()
    for variable in variables:
        data.append(go.Scatter(x = df.index,
                                 y = df[variable],
                                 name=variable))
        
    layout = go.Layout(title='DustTrack vs Honeywell',
                       yaxis=dict(title='Pm 2.5 value'),
                       xaxis=dict(title='N° Observation'))
    fig = go.Figure(data=data, layout=layout)
    plot_url = py.iplot(fig, filename=filename)
    return plot_url
    
# Login plotly
py.sign_in('FranciscoJavierOspinaSalazar', 'WLz5dfnwRjfo2cSwlRiL')

# Visualización inicial de los datos y exploraciones univariadas

In [6]:
plot_time_series(df=df_start, variables=['Valor DustTrack', 'Valor Honeywell'], filename='Comparison_sensors')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~FranciscoJavierOspinaSalazar/0 or inside your plot.ly account where it is named 'Comparison_sensors'


## Boxplot de ambas mediciones

In [7]:
def build_boxplot(df, variables, filename):
    if isinstance(variables, str):
        variables = [variables]
    data = list()
    for variable in variables:
        data.append(go.Box(y = df[variable],
                           name=variable))
        
    layout = go.Layout(title='Boxplot',
                       yaxis=dict(title='Pm 2.5 value'))
    fig = go.Figure(data=data, layout=layout)
    plot_url = py.iplot(fig, filename=filename)
    return plot_url
    

In [8]:
build_boxplot(df=df_start, variables='Valor DustTrack', filename='Boxplot_DustTrack')

In [9]:
build_boxplot(df=df_start, variables='Valor Honeywell', filename='Boxplot_Honeywell')

In [10]:
build_boxplot(df=df_start, variables=['Valor DustTrack', 'Valor Honeywell'], filename='Boxplot_Comparison1')

# Remoción de outliers

In [11]:
df = df_start.copy()
df = df[np.logical_not(np.logical_or(df['Valor Honeywell'] > 90, df['Valor DustTrack'] > 492))]

# Suavizamiento de serie con medias móviles

La descripción de las médias móviles y su uso para suavizar picos en series de tiempo se encuentra en [http://www.expansion.com/diccionario-economico/media-movil.html](http://www.expansion.com/diccionario-economico/media-movil.html)

In [12]:
n_movil_mean = 20
movil_mean = df[['Valor DustTrack', 'Valor Honeywell']].rolling(n_movil_mean).mean()
movil_mean.columns = ['Mm_DustTrack', 'Mm_Honeywell']
movil_mean = movil_mean.loc[n_movil_mean - 1:]

In [13]:
## Reescalamiento de los datos con la función Logaritmo

In [14]:
movil_mean = movil_mean.assign(Log10_DustTrack=np.log10(movil_mean['Mm_DustTrack']))
movil_mean = movil_mean.assign(Log10_Honeywell=np.log10(movil_mean['Mm_Honeywell']))
movil_mean.index = np.arange(len(movil_mean))
movil_mean.head()

,Mm_DustTrack,Mm_Honeywell,Log10_DustTrack,Log10_Honeywell
0,22.60,8.40,1.354108,0.924279
1,22.15,8.25,1.345374,0.916454
2,21.80,8.10,1.338456,0.908485
3,21.55,8.00,1.333447,0.903090
4,21.45,7.90,1.331427,0.897627


In [15]:
plot_time_series(df=movil_mean, variables=['Mm_DustTrack', 'Mm_Honeywell'], filename='serie_movil')

Con el suavizamiento se suprimieron los picos que superaban 2000 en el eje y

In [16]:
plot_time_series(df=movil_mean, variables=['Log10_DustTrack', 'Log10_Honeywell'], filename='log10_serie_movil')

In [17]:
build_boxplot(df=movil_mean, variables=['Log10_DustTrack', 'Log10_Honeywell'], filename='Bloxplot_log10')

# Correlación lineal de Pearson 

[https://es.wikipedia.org/wiki/Coeficiente_de_correlaci%C3%B3n_de_Pearson](https://es.wikipedia.org/wiki/Coeficiente_de_correlaci%C3%B3n_de_Pearson)

Debido a que solo son dos variables si la correlación de pearson es alta se pueden hacer modelos que intenten generar la variable **Valor DustTrack** a partir de **Valor Honeywell**.

In [18]:
# Coeficiente de correlación sobre las variables crudas
# stats.pearsonr(df['Valor Honeywell'], df['Valor DustTrack'])

In [19]:
# Coeficiente de correlación sobre las variables transformadas por logaritmos
# stats.pearsonr(np.log10(df['Valor Honeywell']), np.log10(df['Valor DustTrack']))

In [20]:
# Coeficinte de correlación sobre las series suavizadas por las media móviles
# stats.pearsonr(movil_mean['Mm_DustTrack'], movil_mean['Mm_Honeywell'])

In [21]:
# Coeficinte de correlación sobre las variables transformadas por logaritmos de las
# series suavizadas por las media móviles
stats.pearsonr(movil_mean['Log10_DustTrack'], movil_mean['Log10_Honeywell'])

(0.9770356179506381, 0.0)

Tanto en la gráfica de la serie de tiempo como en la correlación, estas son las mejores variables hasta el momento para el modelo. 

# Construcción de modelos lineales

## Test de shapiro para determinar normalidad en los datos

In [22]:
stats.shapiro(np.random.choice(movil_mean['Log10_DustTrack'], replace=False, size=5000))

(0.9558135867118835, 1.8494479728290665e-36)

## Modelos sin intercepto. De la forma: 
$$y = ax$$

In [23]:
model = sm.OLS(endog=movil_mean['Log10_DustTrack'].values, 
               exog=movil_mean['Log10_Honeywell'].values)
res = model.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.996
Model:                            OLS   Adj. R-squared:                  0.996
Method:                 Least Squares   F-statistic:                 3.403e+06
Date:                Wed, 26 Sep 2018   Prob (F-statistic):               0.00
Time:                        11:29:12   Log-Likelihood:                 8589.1
No. Observations:               14361   AIC:                        -1.718e+04
Df Residuals:                   14360   BIC:                        -1.717e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             1.4557      0.001   1844.666      0.0

In [24]:
print('Test de shapiro sobre los residuales: {}'.format(stats.shapiro(np.random.choice(res.resid,size=5000))[1]))

Test de shapiro sobre los residuales: 2.1978422413341353e-31


## Modelos con intercepto. De la forma: 
$$y = ax + b$$

In [25]:
model2 = sm.OLS(endog=movil_mean['Log10_DustTrack'].values, 
                exog=sm.add_constant(movil_mean['Log10_Honeywell'].values))
res2 = model2.fit()
print(res2.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.955
Model:                            OLS   Adj. R-squared:                  0.955
Method:                 Least Squares   F-statistic:                 3.019e+05
Date:                Wed, 26 Sep 2018   Prob (F-statistic):               0.00
Time:                        11:29:12   Log-Likelihood:                 14763.
No. Observations:               14361   AIC:                        -2.952e+04
Df Residuals:                   14359   BIC:                        -2.951e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4181      0.003    139.883      0.0

In [26]:
print('Test de shapiro sobre los residuales: {}'.format(stats.shapiro(np.random.choice(res.resid,size=5000))[1]))

Test de shapiro sobre los residuales: 9.355045213787479e-30


Tanto la Log-verosimilitud, como el AIC como el BIC muestran que el mejor de los dos modelos es el que tiene intercepto.

## Append de predicciones a las series suavizadas

In [27]:
pred = np.power(10, res.predict(movil_mean['Log10_Honeywell']))
movil_mean = movil_mean.assign(pred_without_intercept=pred)
pred2 = np.power(10, res2.predict(sm.add_constant(movil_mean['Log10_Honeywell'].values)))
movil_mean = movil_mean.assign(pred_with_intercept=pred2)
movil_mean.head()

,Mm_DustTrack,Mm_Honeywell,Log10_DustTrack,Log10_Honeywell,pred_without_intercept,pred_with_intercept
0,22.60,8.40,1.354108,0.924279,22.156422,31.411214
1,22.15,8.25,1.345374,0.916454,21.582813,30.757413
2,21.80,8.10,1.338456,0.908485,21.013939,30.105599
3,21.55,8.00,1.333447,0.903090,20.637342,29.672175
4,21.45,7.90,1.331427,0.897627,20.262884,29.239657


## Append de predicciones a las series sin suaizar

In [28]:
df_start = df_start.assign(pred_without_intercept=np.power(10, res.predict(np.log10(df_start['Valor Honeywell']))))
df_start = df_start.assign(pred_with_intercept=np.power(10, res2.predict(sm.add_constant(np.log10(df_start['Valor Honeywell'])))))
df_start.head()

,Valor DustTrack,Valor Honeywell,Hora,pred_without_intercept,pred_with_intercept
0,26,10,07:06:26,28.558041,38.501470
1,26,10,07:06:27,28.558041,38.501470
2,26,9,07:06:28,24.497284,34.045702
3,25,9,07:06:29,24.497284,34.045702
4,22,9,07:06:30,24.497284,34.045702


# Plots del valor real vs la predicción del modelo (sobre serie suavizada)

In [29]:
plot_time_series(df=movil_mean, 
                 variables=['Mm_DustTrack', 'pred_with_intercept'], 
                 filename='Predictions_01')

In [30]:
plot_time_series(df=df_start,
                 variables=['Valor DustTrack', 'pred_with_intercept'],
                 filename='Predictions_02')

# Errores de la predicción vs el valor real

In [31]:
# Error cuadratico medio para el modelo con intercepto
mean_squared_error(df_start['Valor DustTrack'], df_start['pred_with_intercept'])

3182.2668604078153

In [32]:
# Error absoluto medio para el modelo con intercepto

mean_absolute_error(df_start['Valor DustTrack'], df_start['pred_with_intercept'])

26.18394928335687

In [33]:
# Error cuadratico medio para el modelo sin intercepto

mean_squared_error(df_start['Valor DustTrack'], df_start['pred_without_intercept'])

9296.27726285298

In [34]:
# Error absoluto medio para el modelo con intercepto

mean_absolute_error(df_start['Valor DustTrack'], df_start['pred_without_intercept'])

46.155642698282406

# Ecuación final de modelamiento

La ecuación final que mejor se comporta para el modelamiento de los valores de **Valor DustTrack** a partir de los datos de **Valor Honeywell** es:

$$y = 10^{1.1674 * log_{10}(x) + 0.4181}$$


Donde X es el valor de la variable **Valor Honeywell**  y y el valor de la variable **Valor DustTrack** 

# Conclusiones finales

- Es posible hacer una reconstrucción no perfecta pero si bastante buena de los datos del sensor **DustTrack** con base en los datos del sensor **Honeywell**.

- Puede haber nuevas transformaciones de los datos que permitan tener modelamietos más precisos. Transformar mediante el logaritmo funciona bien pero eso no quiere decir que no pueda haber mejores transformaciones.

- Queda pendiente la evaluación de nuevos modelos (No lineales).

- Queda pendiente la evaluación de modelos basados en ML.